# Jason Isbell Concert Journey Analysis
## 8 Shows • 154 Songs • 2022-2025

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from concert_analysis import JasonIsbellConcertData

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Load the data
concerts = JasonIsbellConcertData('data/setlists.csv')
df = concerts.df

# Display basic stats
stats = concerts.get_stats()
print(f"Total Shows: {stats['total_shows']}")
print(f"Total Songs: {stats['total_songs']}")
print(f"Unique Songs: {stats['unique_songs']}")
print(f"Date Range: {stats['date_range']['first_show']} to {stats['date_range']['last_show']}")

## Most Played Songs

In [ ]:
# Get top 10 songs
top_songs = concerts.get_top_songs(10)

# Create visualization
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(top_songs['clean_song'], top_songs['play_count'])
ax.set_xlabel('Number of Shows')
ax.set_title('Top 10 Most Played Songs')
ax.invert_yaxis()

# Add percentage labels
for i, (count, pct) in enumerate(zip(top_songs['play_count'], top_songs['percentage'])):
    ax.text(count + 0.1, i, f'{pct}%', va='center')

plt.tight_layout()
plt.show()

## Songs Per Show Timeline

In [ ]:
# Songs per show
songs_per_show = df.groupby('Date').size().reset_index(name='song_count')
songs_per_show = songs_per_show.sort_values('Date')

# Create timeline
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(songs_per_show['Date'], songs_per_show['song_count'], 'o-', markersize=10)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Songs')
ax.set_title('Songs Per Show Over Time')
ax.grid(True, alpha=0.3)

# Add venue labels
for _, row in songs_per_show.iterrows():
    venue = df[df['Date'] == row['Date']]['Venue'].iloc[0].split(' - ')[0]
    ax.annotate(venue, (row['Date'], row['song_count']), 
                textcoords='offset points', xytext=(0,10), 
                ha='center', fontsize=8, rotation=45)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Venue Analysis

In [ ]:
# Venue statistics
venue_stats = concerts.get_venue_stats()
print(venue_stats)

## Cover Songs Analysis

In [ ]:
# Analyze covers
covers = df[df['is_cover']]
print(f"Total cover performances: {len(covers)}")
print(f"\nUnique covers: {covers['clean_song'].nunique()}")
print("\nMost performed covers:")
print(covers['clean_song'].value_counts().head(10))

## Rare Songs (Played Only Once)

In [ ]:
# Get rare songs
rare_songs = concerts.get_rare_songs()
print(f"Songs played only once: {len(rare_songs)}")
print("\nRare gems:")
for song in sorted(rare_songs)[:10]:
    print(f"  • {song}")

## Export for Further Analysis

In [ ]:
# Export complete data
import json

viz_data = concerts.export_for_visualization()
with open('concert_data.json', 'w') as f:
    json.dump(viz_data, f, indent=2, default=str)

print("Data exported to concert_data.json")
print(f"\nExported data includes:")
print(f"  • {len(viz_data['shows'])} shows with complete setlists")
print(f"  • Top {len(viz_data['song_frequency'])} songs with play counts")
print(f"  • Venue statistics")
print(f"  • {len(viz_data['rare_songs'])} rare songs")